## Implementing Random Forrest from Scratch 


In [6]:
# importing required libraries 

import pandas as pd 
import numpy as np 
import random 
import copy

In [17]:
class DecisionTree():
    def __init__(self, df, column_types , min_samples=10, max_depth=5 , random_features = None):
        
        self.max_depth = max_depth 
        self.min_samples = min_samples
        self.random_features = random_features
        self.data = df.values
        self.feature_names = df.columns
        self.feature_types = column_types
        self.tree = self.DT_algorithm(self.data, self.min_samples, self.max_depth , self.random_features)
        
           
    
    def find_candidate_splits(self, data , random_features):
    
        splits = {}
        col_indices = list(range(data.shape[1] - 1))            # excluding the last column which is the label

        if (random_features > 0)  and (random_features <= len(col_indices)):
                col_indices = random.sample(col_indices , random_features)

        for col_index in col_indices:                         
            unique_values = np.unique(data[:, col_index])

            type_of_feature = self.feature_types[col_index]
            if type_of_feature == "numerical":
                if len(unique_values) > 0 :
                    splits[col_index] = []
                    for index in range(len(unique_values)):
                        if index != 0:

                            split = (unique_values[index] + unique_values[index - 1]) / 2
                            splits[col_index].append(split)

            elif (type_of_feature == "categorical") and (len(unique_values) > 1):
                splits[col_index] = unique_values


        return splits
    
    
    
    def find_best_attribute(self , data, candidate_splits):
    
        least_mse_value = float('inf')
        for col_index in candidate_splits:
            for value in candidate_splits[col_index]:
                left_hand_split, right_hand_split = self.split_samples_func(data, col_index, value)
                overall_mse_error = self.overal_mse_error(left_hand_split, right_hand_split)

                if overall_mse_error <= least_mse_value:
                    
                    least_mse_value = overall_mse_error
                    best_split_column = col_index
                    best_split_value = value

        return best_split_column, best_split_value
    
    
    
    def split_samples_func(self , data, split_column, split_value):
    
        split_column_values = data[:, split_column]
        type_of_feature = self.feature_types[split_column]
        
        if type_of_feature == "numerical":
            left_hand_split = data[split_column_values <= split_value]
            right_hand_split = data[split_column_values >  split_value]

        # feature is categorical   
        else:
            left_hand_split = data[split_column_values == split_value]
            right_hand_split = data[split_column_values != split_value]

        return left_hand_split, right_hand_split
    
    
    def mean_square_error(self , data):
        price_values = data[:, -1]
        if len(price_values) == 0:   
            m_s_error = 0

        else:
            mean_price = np.mean(price_values)
            m_s_error = np.mean((price_values - mean_price) **2)

        return m_s_error
    
    
    def overal_mse_error(self , left_hand_split, right_hand_split):

        total_node_samples = len(left_hand_split) + len(right_hand_split)
        p_left_hand_split = len(left_hand_split) / total_node_samples
        p_right_hand_split = len(right_hand_split) / total_node_samples

        overall_mse =  (p_left_hand_split *self.mean_square_error(left_hand_split)+ p_right_hand_split * self.mean_square_error(right_hand_split)) 

        return overall_mse
    
    
    
    
    def DT_algorithm(self , data, min_samples, max_depth, random_features, current_depth=0):
        
                    
        if (self.mean_square_error(data) == 0) or (len(data) < min_samples) or (current_depth == max_depth):
            leaf = np.mean(data[:, -1])
            return leaf

           
        else:    
            current_depth += 1

            candidate_splits = self.find_candidate_splits(data , random_features)
            if len(candidate_splits)== 0 :
                leaf = np.mean(data[:, -1])
                return leaf
            
            split_column, split_value = self.find_best_attribute(data, candidate_splits)
            left_hand_split, right_hand_split = self.split_samples_func(data, split_column, split_value)

            # check for empty data
            if len(left_hand_split) == 0 or len(right_hand_split) == 0:
                leaf = np.mean(data[:, -1])
                return leaf

            # determine node_condition
            feature_name = self.feature_names[split_column]
            type_of_feature = self.feature_types[split_column]
            
            # feature is numerical
            if type_of_feature == "numerical":
                node_condition = "{} <= {}".format(feature_name, split_value)

            # feature is categorical
            else:
                node_condition = "{} = {}".format(feature_name, split_value)

            # instantiate sub-tree
            sub_tree = {node_condition: []}

            # find answers (recursion)
            positive_answer = self.DT_algorithm(left_hand_split, min_samples, max_depth , random_features , current_depth)
            negative_answer = self.DT_algorithm(right_hand_split,min_samples, max_depth , random_features , current_depth)

            if positive_answer == negative_answer:
                sub_tree = positive_answer
            else:
                sub_tree[node_condition].append(positive_answer)
                sub_tree[node_condition].append(negative_answer)

            return sub_tree
     
    
    
    def predict_sample(self , sample):
        
        tree = self.tree.copy()
        while (isinstance(tree , dict)):

            node_condition = list(tree.keys())[0]
            feature_name, operator, value = node_condition.split(" ")

            # feature is numerical
            if operator == "<=":
                if sample[feature_name] <= float(value):
                    sub_tree = tree[node_condition][0]
                else:
                    sub_tree = tree[node_condition][1]

            # feature is categorical
            else:
                if str(sample[feature_name]) == value:
                    sub_tree = tree[node_condition][0]
                else:
                    sub_tree = tree[node_condition][1]

            tree = sub_tree 


        return tree
        

In [8]:
def random_forest_algorithm(train_df, column_types, n_trees, n_bootstrap, n_features, dt_max_depth, dt_min_samples):
    forest = []
    for i in range(n_trees):
        
        bootstrap_indices = np.random.choice(list(range(len(train_df))), size=n_bootstrap , replace=True)
        df_bootstrapped = train_df.iloc[bootstrap_indices]
        tree = DecisionTree(df_bootstrapped, column_types ,min_samples=dt_min_samples, max_depth=dt_max_depth, random_features=n_features)
        forest.append(tree)
    
    return forest


def random_forest_predictions(test_df, forest):

    rf_predictions = []
    for i in range(len(test_df)):
        predictions = [tree.predict_sample(test_df.iloc[i]) for tree in forest]
        rf_predictions.append(np.mean(predictions))
        
    return rf_predictions

In [9]:
def data_loading(path):
    
    df = pd.read_csv(path)
    df = df.drop(['PoolQC','MiscFeature','Alley','Fence','FireplaceQu'] , axis=1)
    df['LotFrontage'] = df['LotFrontage'].fillna(int(df['LotFrontage'].mean()))
    df = df.set_index('Id')
    
    for i in range(len(df.columns)):
        if df[df.columns[i]].isnull().sum():
            df[df.columns[i]] = df[df.columns[i]].fillna(df[df.columns[i]].mode()[0])
            
        if df[df.columns[i]].dtype == object:
            df[df.columns[i]]= df[df.columns[i]].apply(lambda x :  x.replace(" ", ""))
          
       
       
    return df

In [15]:
def get_column_type(df):
    
    column_types = [] 
    max_values = 30
    for feature in df.columns:
        if feature != "SalePrice":
            values = df[feature].unique()

            if (isinstance(values[0], str)) or (len(values) <= max_values):
                column_types.append("categorical")
            else:
                column_types.append("numerical")
                
    return column_types
        

In [11]:
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

In [18]:
def main():
    
    df_train = data_loading('./data/housing_price_train.csv')
    df_test = data_loading('./data/housing_price_test.csv')
    column_types = get_column_type(df_train)
    forest = random_forest_algorithm(df_train, column_types, n_trees=250, n_bootstrap=1500, n_features=30, dt_max_depth=5 , dt_min_samples=15)
    predictions = random_forest_predictions(df_test, forest)
    createSubmission(df_test.index , predictions)

if __name__ == "__main__":
    main()